# Retrieval Augmented Generation (RAG) with Actions

Using actions in NeMo Guardrails gives us the ability to do a lot of things without needing heavy agent decision making pipelines. We can enable to use of tools with little more than what is essentially a "fuzzy logic match" between a users input and the intent groups that we define in our Colang files.

In this example, we'll be taking a look at how to apply the power of **R**etrieval **A**ugmented **G**eneration (RAG) to LLMs using nothing more than Colang logic.

We'll get started by installing the prerequisite libraries:

In [ ]:
!pip install -qU \
    nemoguardrails==0.4.0 \
    pinecone-client==2.2.2 \
    datasets==2.14.3 \
    openai==0.27.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 33.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

## Knowledge Base Download

To begin, we need to setup our data and retrieval components for RAG. We'll start with a dataset that contains info on the recent Llama 2 models:

In [ ]:
from datasets import load_dataset

data = load_dataset(
    "jamescalam/llama-2-arxiv-papers-chunked",
    split="train"
)
data

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references'],
    num_rows: 4838
})

In [ ]:
data[0]

{'doi': '1102.0183',
 'chunk-id': '0',
 'chunk': 'High-Performance Neural Networks\nfor Visual Object Classi\x0ccation\nDan C. Cire\x18 san, Ueli Meier, Jonathan Masci,\nLuca M. Gambardella and J\x7f urgen Schmidhuber\nTechnical Report No. IDSIA-01-11\nJanuary 2011\nIDSIA / USI-SUPSI\nDalle Molle Institute for Arti\x0ccial Intelligence\nGalleria 2, 6928 Manno, Switzerland\nIDSIA is a joint institute of both University of Lugano (USI) and University of Applied Sciences of Southern Switzerland (SUPSI),\nand was founded in 1988 by the Dalle Molle Foundation which promoted quality of life.\nThis work was partially supported by the Swiss Commission for Technology and Innovation (CTI), Project n. 9688.1 IFF:\nIntelligent Fill in Form.arXiv:1102.0183v1  [cs.AI]  1 Feb 2011\nTechnical Report No. IDSIA-01-11 1\nHigh-Performance Neural Networks\nfor Visual Object Classi\x0ccation\nDan C. Cire\x18 san, Ueli Meier, Jonathan Masci,\nLuca M. Gambardella and J\x7f urgen Schmidhuber\nJanuary 2011\nAbs

We mainly want the information contained within the `chunk` parameter, although we can pull in other bits of data as metadata for use later. We'll also create a new unique ID for each record by concatenating the `doi` and `chunk-id` fields.

In [ ]:
data = data.map(lambda x: {
    'uid': f"{x['doi']}-{x['chunk-id']}"
})
data

Map:   0%|          | 0/4838 [00:00<?, ? examples/s]

Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'id', 'title', 'summary', 'source', 'authors', 'categories', 'comment', 'journal_ref', 'primary_category', 'published', 'updated', 'references', 'uid'],
    num_rows: 4838
})

In [ ]:
data = data.to_pandas()
# drop irrelevant fields
data = data[['uid', 'chunk', 'title', 'source']]

`chunk` will be the text that we encode and store inside Pinecone. To encode that text data we need to use an embedding model, for that we can use open source [sentence transformers](https://github.com/UKPLab/sentence-transformers), Cohere, OpenAI, and many other services. In this example we will use OpenAI, to do so we will need an [OpenAI API key](https://platform.openai.com) (there will be some minor embedding cost incurred here).

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-PYP-----"

Now we can create embeddings like so:

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
import openai

embed_model_id = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "We would have some text to embed here",
        "And maybe another chunk here too"
    ], engine=embed_model_id
)

In [ ]:
res.keys()

dict_keys(['object', 'data', 'model', 'usage'])

Inside the response `res` we will find a JSON like object containing our new embeddings within the `data` field:

In [ ]:
len(res['data'])

2

In [ ]:
len(res['data'][0]['embedding']), len(res['data'][1]['embedding'])

(1536, 1536)

Each embedding has a dimensionality of `1536`, as this is the embedding dimensionality of the `text-embedding-ada-002` model. We will apply this same embedding logic to the dataset we downloaded before, but before doing so we must create a vector DB index where we can store those embeddings.

## Creating the Knowledge Base

Now we need a place to store these embeddings and enable a efficient vector search through them all. To do that we use Pinecone, we can get a [free API key](https://app.pinecone.io/) and enter it below where we will initialize our connection to Pinecone and create a new index.

In [ ]:
!pip3 install pinecone-client

In [ ]:
!poetry add pinecone

/bin/bash: line 1: poetry: command not found


In [ ]:
import os
from pinecone.grpc import PineconeGRPC
# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("4624ddfa-25c2-4bff-b314-c2de1104ac17")
# find your environment next to the api key in pinecone console
env = os.getenv("PINECONE_ENVIRONMENT") or "YOUR_ENV"

pc = Pinecone(api_key=api_key)
pinecone.whoami()

ModuleNotFoundError: No module named 'pinecone.grpc'

Create the index:

In [ ]:
index_name = "nemo-guardrails-rag-with-actions"

In [ ]:
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes().names():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='cosine'
    )
    # wait for index to be initialized
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pinecone.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 4838}},
 'total_vector_count': 4838}

We can see the index is currently empty with a `total_vector_count` of `0`. We can begin populating it with OpenAI `text-embedding-ada-002` embeddings like so:

In [ ]:
from tqdm.auto import tqdm

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(data), batch_size)):
    # find end of batch
    i_end = min(len(data), i+batch_size)
    batch = data[i:i_end]
    # get ids
    ids_batch = batch['uid'].to_list()
    # get texts to encode
    texts = batch['chunk'].to_list()
    # create embeddings
    res = openai.Embedding.create(input=texts, engine=embed_model_id)
    embeds = [record['embedding'] for record in res['data']]
    # create metadata
    metadata = [{
        'chunk': x['chunk'],
        'source': x['source']
    } for _, x in batch.iterrows()]
    to_upsert = list(zip(ids_batch, embeds, metadata))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 0/49 [00:00<?, ?it/s]

## RAG Functions for Guardrails

Now that we've added all of our text data to the index let's create a "retrieve function" that will allow us to take a user query, retrieve relevant records, and return them for use by our LLM.

_Note: all functions defined and used with Guardrails `generate_async` must also be async functions._

In [ ]:
async def retrieve(query: str) -> list:
    # create query embedding
    res = openai.Embedding.create(input=[query], engine=embed_model_id)
    xq = res['data'][0]['embedding']
    # get relevant contexts from pinecone
    res = index.query(xq, top_k=5, include_metadata=True)
    # get list of retrieved texts
    contexts = [x['metadata']['chunk'] for x in res['matches']]
    return contexts

We will create another function to perform the actual retrieval augmented generation (RAG) step given a particular query and contexts.

In [ ]:
async def rag(query: str, contexts: list) -> str:
    print("> RAG Called")  # we'll add this so we can see when this is being used
    context_str = "\n".join(contexts)
    # place query and contexts into RAG prompt
    prompt = f"""You are a helpful assistant, below is a query from a user and
    some relevant contexts. Answer the question given the information in those
    contexts. If you cannot find the answer to the question, say "I don't know".

    Contexts:
    {context_str}

    Query: {query}

    Answer: """
    # generate answer
    res = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0.0,
        max_tokens=100
    )
    return res['choices'][0]['text']

## Guardrails

We now need to initialize our configs for Rails:

In [ ]:
yaml_content = """
models:
- type: main
  engine: openai
  model: text-davinci-003
"""

rag_colang_content = """
# define limits
define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot answer politics
    "I'm a shopping assistant, I don't like to talk of politics."
    "Sorry I can't talk about politics!"

define flow politics
    user ask politics
    bot answer politics
    bot offer help

# define RAG intents and flow
define user ask llama
    "tell me about llama 2?"
    "what is large language model"
    "where did meta's new model come from?"
    "how to llama?"
    "have you ever meta llama?"

define flow llama
    user ask llama
    $contexts = execute retrieve(query=$last_user_message)
    $answer = execute rag(query=$last_user_message, contexts=$contexts)
    bot $answer
"""

Note how we have created a user message (`user ask llama`) and flow (`llama`) for handling user queries if they ask about anything Llama 2 / LLM related:

```colang
define user ask llama
    "tell me about llama 2?"
    "what is large language model"
    "where did meta's new model come from?"
    "how to llama?"
    "have you ever meta llama?"

define flow llama
    user ask llama
    $contexts = execute retrieve(query=$last_user_message)
    $answer = execute rag(query=$last_user_message, contexts=$contexts)
    bot $answer
```

It executes the `retrieve` action using the `$last_user_message` to get our `$contexts`, we then pass the `$last_user_message` and `$contexts` to our `rag` action. We initialize our RAG-enabled rails with this Colang setup:

In [ ]:
from nemoguardrails import LLMRails, RailsConfig

# initialize rails config
config = RailsConfig.from_content(
    colang_content=rag_colang_content,
    yaml_content=yaml_content
)
# create rails
rag_rails = LLMRails(config)

Remember! We need to register any actions that are used in the Colang config file, otherwise our rails have no idea how to `execute retrieve` or `execute rag`. We register both like so:

In [ ]:
rag_rails.register_action(action=retrieve, name="retrieve")
rag_rails.register_action(action=rag, name="rag")

Now let's try out our RAG agent.

In [ ]:
await rag_rails.generate_async(prompt="hello")

'Hi there! How can I help you today?'

In [ ]:
await rag_rails.generate_async(prompt="tell me about llama 2")

> RAG Called


' Llama 2 is a collection of pretrained and fine-tuned large language models (LLMs) ranging in scale from 7 billion to 70 billion parameters. They are optimized for dialogue use cases and outperform open-source chat models on most benchmarks. They are also on par with some closed-source models, at least on the human evaluations performed. They are intended for commercial and research use in English and can be adapted for a variety of natural language generation tasks.'

We can see from the printed statement of `> RAG Called` that the RAG pipeline was used in our second prompt. However, it'd be interesting to see what the effect of this pipeline is on our actual answer. To do that, let's initialize a new Rails instance that doesn't include the call to RAG, so we can compare the results:

In [ ]:
no_rag_colang_content = """
# define limits
define user ask politics
    "what are your political beliefs?"
    "thoughts on the president?"
    "left wing"
    "right wing"

define bot answer politics
    "I'm a shopping assistant, I don't like to talk of politics."
    "Sorry I can't talk about politics!"

define flow politics
    user ask politics
    bot answer politics
    bot offer help
"""

In [ ]:
# initialize rails config
config = RailsConfig.from_content(
    colang_content=no_rag_colang_content,
    yaml_content=yaml_content
)
# create rails
no_rag_rails = LLMRails(config)

Let's ask the same question:

In [ ]:
await no_rag_rails.generate_async(prompt="tell me about llama 2")

"Llama 2 is a text-to-speech software developed by NVIDIA. It is designed to generate natural sounding speech from text and is used in a variety of applications such as virtual assistants, chatbots, and automated customer service. The software is powered by NVIDIA's AI platform and uses a deep learning model to generate the audio output."

Hmm, not the Llama 2 we were looking for — let's ask some more questions and compare RAG vs. no-RAG answers.

In [ ]:
# no RAG
await no_rag_rails.generate_async(
    prompt="what was red teaming used for in llama 2 training?"
)

'Red teaming was used in Llama 2 training to simulate enemy tactics and techniques. This allowed the trainees to practice dealing with realistic threats and build strategies to counter them.'

Our no RAG rails provide an interesting, but completely wrong answer. Let's try the same with our RAG-enabled rails:

In [ ]:
# with RAG
await rag_rails.generate_async(
    prompt="what was red teaming used for in llama 2 training?"
)

> RAG Called


' Red teaming was used to identify risks and to measure the robustness of the model with respect to a red teaming exercise executed by a set of experts. It was also used to provide qualitative insights to recognize and target specific patterns in a more comprehensive way.'

A perfect answer! Clearly, our RAG-enabled rail is far more capable of answering questions, while only calling the RAG action when required as set in our `actions.co` config file. We can confirm by asking more questions, we should see the printed statement `"> RAG Called"` will not appear unless the question is Llama 2 / LLM related:

In [ ]:
await rag_rails.generate_async(
    prompt="what color is the sky?"
)

'The sky is typically blue, but can change depending on atmospheric conditions and time of day.'

By using Guardrails for RAG in this way we manage to create a balance between the lightweight but naive approach of implementing [RAG with _every_ user call](https://www.pinecone.io/learn/series/langchain/langchain-retrieval-augmentation/) vs. the heavyweight and slow approach of implementing a [conversational agent with RAG tool access](https://www.pinecone.io/learn/series/langchain/langchain-agents/).